<H2>Composite steel girder deck</H2>

In [ ]:
'''Inputs - Units are N,mm'''
segments_lengths = [6000, 5000, 6000, 3000, 3000, 6000, 5000, 6000]
index_support = 4

mesh_size = 500

transverse_spacing = 5000
deck_width = 7000
deck_thk = 200

Create a girder class to hold the section dimensions for each segment</br>
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class GirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Create a list of section definitions, we'll need one per segment + 1 for the end.

In [ ]:
sections1 = []
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1800, 20, 600, 50))
sections1.append(GirderSection(500, 30, 1800, 20, 600, 50)) # Central support
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections1)-1)
import copy
sections2 = [copy.deepcopy(s) for s in sections1]

sections_list = [sections1, sections2]

Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [ ]:
stiffener_thicknesses = [20, 10, 10, 20, 25, 20, 10, 10, 20]

# Ensure number of stiffener thicknesses matches the number of segments
assert(len(sections1) == len(stiffener_thicknesses))

Connect to LUSAS Modeller

In [ ]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Composite Steel Bridge Deck")
else:
    import sys; sys.path.append('../') # Locate the helpers file
    from m100_Tools_And_Helpers import Helpers
    Helpers.initialise(lusas)
    Helpers.reset_database_contents(lusas.database())

db = lusas.database() # Get a reference to the current model database for convenience

# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")

# Units N,mm
db.setModelUnits("N,mm,kg,s,C")


Create model attributes

In [ ]:
# Create the mesh attribute
shell_mesh_attr = db.createMeshSurface(f"Shell Mesh {mesh_size}mm").setRegularSize("QTS4", mesh_size, True)

# Bar mesh attributes
bracing_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("Bar", 1)

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8, 10e-6)

# Concrete Material attribute
concrete_material = db.createIsotropicMaterial("Concrete", 30_000, 0.2, 2.4, 10e-6)

# Support attributes
pinned = db.createSupportStructural("Fixed in XYZ").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in YZ").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")

Define a helper function to create a point in the model.</br>The db.createPoint() function returns an IFObjectSet which can contain multiple objects.</br>Since we have only created a single point we'll get a reference to the point and return it

In [ ]:
def create_point(x:float, y:float, z:float) -> 'IFPoint':
    return win32.CastTo(db.createPoint(x, y, z).getObjects("Point")[0], "IFPoint")

Create all the points for each section, saving the returned point in the Girder section definition

In [ ]:
s:GirderSection
for sections, y in [(sections1, -transverse_spacing/2),(sections2, transverse_spacing/2)]:
    x = 0
    for i in range(0, len(sections)):
        # Determine the longitudinal position (x coord) for each section
        if i > 0 : x += segments_lengths[i-1]
        # Determine the section to use at each longitudinal position
        s = sections[i]
        # Create the bottom flange points
        s.bf_points.append(create_point(x, y + s.bottom_flange_breadth/2, -s.web_depth))
        s.bf_points.append(create_point(x, y,                             -s.web_depth))
        s.bf_points.append(create_point(x, y - s.bottom_flange_breadth/2, -s.web_depth))
        # Create the top flange points
        s.tf_points.append(create_point(x, y +s.top_flange_breadth/2, 0.0))
        s.tf_points.append(create_point(x, y,                         0.0))
        s.tf_points.append(create_point(x, y -s.top_flange_breadth/2, 0.0))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface(pnts: list, thk:float, ecc:float, group:str):

    geometryData = lusas.geometryData().setAllDefaults()
    geometryData.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet().add(pnts)

    surfaces = obs.createSurface(geometryData)
    db.getGroupByName(group).add(surfaces, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(surfaces)

In [ ]:
def create_line(p1: 'IFPoint', p2: 'IFPoint'):

    geometryData = lusas.geometryData().setAllDefaults()
    geometryData.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet().add(p1).add(p2)

    return obs.createLine(geometryData).getObjects("Line")[0]

Create groups to contain the girder surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")
db.createEmptyGroup("Deck")

Here we'll create the surfaces between each section definition using the points we created earlier

In [ ]:
for sections in sections_list:
    for i in range(0, len(segments_lengths)):

        s1:GirderSection = sections[i]   # Section definition at the start of the segment
        s2:GirderSection = sections[i+1] # Section definition at the end of the segment

        create_surface([ s1.bf_points[0], s2.bf_points[0], s2.bf_points[1], s1.bf_points[1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[2], s1.bf_points[2] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[1], s1.tf_points[1], s2.tf_points[1], s2.bf_points[1] ], s1.web_thk,           0.0, "Web")
        create_surface([ s1.tf_points[0], s2.tf_points[0], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")
        create_surface([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[2], s1.tf_points[2] ], s1.top_flange_thk,    0.0, "Top Flange")


Create bearing stiffeners at the end sections

In [ ]:
for sections in sections_list:
    for i, thk in enumerate(stiffener_thicknesses):

        if thk > 0:

            s:GirderSection = sections[i]   # Section definition at the start of the segment
            create_surface([ s.bf_points[0], s.tf_points[0], s.tf_points[1], s.bf_points[1] ], thk, 0.0, "Web Stiffeners")
            create_surface([ s.bf_points[1], s.tf_points[1], s.tf_points[2], s.bf_points[2] ], thk, 0.0, "Web Stiffeners")


Assign a shell mesh attribute and steel material attribute to all surfaces

In [ ]:
# Assign to all 
shell_mesh_attr.assignToAll()
steel_material.assignToAll()

Support attributes

In [ ]:
# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have created for each section

# Do this for the start and end sections
for i, section in enumerate([sections[0], sections[index_support], sections[-1]]):

    p0 = section.bf_points[0] # point at the +ve flange edge
    p1 = section.bf_points[1] # point at the centre of the web.
    p2 = section.bf_points[2] # point at the -ve flange edge

    # Loop through connected lines 
    for line in p1.getHOFs():
        line = win32.CastTo(line, "IFLine")
        # Points of the connected line
        ps1 = line.getStartPoint()
        ps2 = line.getEndPoint()
        # Check if the connected line points are those defining the bottom flange
        # If so then we can assign the relevant support attribute
        if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
            pinned.assignTo(line) if i == 1 else slide.assignTo(line)


In [ ]:

db.setDefault("Mesh", bracing_mesh_attr)

db.setDefault("Material", steel_material)



In [ ]:
#bracing_lines = []
for i in range(0, segments_lengths):
    line1 = create_line(sections1[i].bf_points[0], sections2[i].tf_points[2])
    line2 = create_line(sections1[i].tf_points[2], sections2[i].bf_points[0])
    lusas.newObjectSet().add(line1).add(line2).splitLines()

    line3 = create_line(sections1[i].bf_points[2], sections2[i].bf_points[0])
    line4 = create_line(sections1[i].tf_points[2], sections2[i].tf_points[0])

# Slab Deck

In [ ]:
x_grid = [0]
for s in segments_lengths:
    x_grid.append(x_grid[-1]+s)

y_grid = [-deck_width/2, -transverse_spacing/2, 0.0, transverse_spacing/2, deck_width/2]

In [ ]:
# Create the actual points in the model
deck_points = [[create_point(x, y, deck_thk/2) for x in x_grid] for y in y_grid]
# [Y][X]

# Create the surfaces for the deck
for iy in range(0, len(y_grid)-1):
    for ix in range(0, len(x_grid)-1):
        create_surface([ deck_points[iy][ix], deck_points[iy][ix+1], deck_points[iy+1][ix+1], deck_points[iy+1][ix] ], deck_thk, 0.0, "Deck")

In [ ]:
shell_mesh_attr.assignTo("Group", "Deck")
concrete_material.assignTo("Group", "Deck")

# Deck to Girder connection

Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Buckling Analysis", False)

# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Buckling", analysis.getName())
# Set the loadcase controls to be a Buckling analysis searching for the minimum 5 buckling modes
first_loadcase.setEigenvalueMaxMinControl("Buckling", "Minimum", 5)


In [ ]:
load_attr = db.createLoadingGlobalDistributed("Global Distributed Load")
load_attr.setGlobalDistributed("Length", 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)


In [ ]:
# Type definiton
gs:GirderSection 

# List of the lines in the top flange
centre_lines = []

# Select the centrelines of the steel girder top flanges
for setions in sections_list:
    for i in range(0, len(sections)):
        gs = sections[i]
        point = gs.tf_points[1]

        for hof in point.getHOFs():
            if hof.getTypeCode() == 2 : # Line in the top flange
                
                line = win32.CastTo(hof, "IFLine")
                
                # Check it is aligned with and paralle to the x axis
                if abs(line.getEndPosition()[1] - line.getStartPosition()[1]) < 1e-3 and \
                    abs(line.getEndPosition()[2] - line.getStartPosition()[2]) < 1e-3:

                    centre_lines.append(line)


load_attr.assignTo(centre_lines, lusas.assignment().setAllDefaults().setLoadset(first_loadcase))



In [ ]:
# Mesh the model
db.resetMesh()
db.updateMesh()

In [ ]:
# Solve all the anlyses
db.getAnalysis("Analysis 1").solve(True)
db.getAnalysis("Eigenvalue Buckling Analysis").solve(True)
db.openAllResults(False)


In [ ]:
loadset = db.getLoadset("Mode 1", 2)
lusas.view().setActiveLoadset(loadset)
